# RAG Application - POC

### Enter your query

In [88]:
query_text = input("what is NICC full name?")
query_text

'"what is NICC full name?"'

In [1]:
!pwd


/root/phi3-rag-application


### Import LangChain libraries

In [67]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.llms import Ollama

### Import utility functions

In [68]:
from keyword_generator import extract_keywords
from db import get_db_collection, add_to_collection, query_collection

## Load pdf document and load it into Vector Database

In [69]:
file_path = (
    "docs/wildfire_stats.pdf"
)
loader = PyPDFLoader(file_path)
document = loader.load()
print("No. of pages in the document:", len(document))

No. of pages in the document: 3


#### Split pages into chunks of texts

In [70]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=50000, chunk_overlap=5000)
chunked_documents = text_splitter.split_documents(document)

#### Prepare data for indexing
- Generate Unique Id for individual chunks
- Generate keywords for metadata using NLP

In [71]:
contents = []
ids = []
keywords = []

page_no = 0
c_index = -1
for index, doc in enumerate(chunked_documents):
    metadata = doc.metadata
    source = metadata['source'].replace('/','-').replace('.','-')

    if metadata['page'] > page_no:
        c_index = 0
    else:
        c_index += 1

    page_no = metadata['page']
    
    chunk_id = f"{source}-p{page_no}-c{c_index}"

    contents.append(doc.page_content)
    ids.append(chunk_id)
    keywords.append(extract_keywords(doc.page_content))
    print("Processed chunk:", chunk_id)

Processed chunk: docs-wildfire_stats-pdf-p0-c0
Processed chunk: docs-wildfire_stats-pdf-p1-c0
Processed chunk: docs-wildfire_stats-pdf-p2-c0


### Create a collection in Chroma DB

In [72]:
COLLECTION_NAME = "my_project3"
collection = get_db_collection(COLLECTION_NAME)

metadata = [{"tags": ", ".join(i) } for i in keywords]
add_to_collection(collection, contents, ids, metadata)

Collection my_project3 does not exist.
Documents loaded to DB


### Chunks retreived from the DB

In [89]:
query_result = query_collection(collection, query_text)
query_result

{'ids': [['docs-wildfire_stats-pdf-p2-c0',
   'docs-wildfire_stats-pdf-p0-c0',
   'docs-wildfire_stats-pdf-p1-c0']],
 'distances': [[0.5116881475005692, 0.6069862841520937, 0.6091022732249572]],
 'metadatas': [[{'tags': 'crs, congress, may, report, congressional'},
   {'tags': 'acres, wildfires, fires, burned, annual'},
   {'tags': 'burned, acres, wildfires, acreage, million'}]],
 'embeddings': None,
 'documents': [['Wildfire Statistics  \nhttps://crsreports.congress.gov  | IF10244  · VERSION 68 · UPDATED   \n \nDisclaimer  \nThis document was  prepared by the Congressional Research Service (CRS). CRS serves as nonpartisan shared staff to \ncongressional committees and Members of Congress. It operates solely at the behest of and under the direction of Congress. \nInformation in a CRS Report should n ot be relied upon for purposes other than public understanding of information that has \nbeen provided by CRS to Members of Congress in connection with CRS’s institutional role. CRS Reports

### Prepare final prompt to give to LLM model

In [90]:
text = ""
for doc in query_result['documents']:
    for i in doc:
        text += i

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
).format(context=text)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
final_prompt = prompt.format(input=query_text)
final_prompt

'System: You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don\'t know the answer, say that you don\'t know. Use three sentences maximum and keep the answer concise.\n\nWildfire Statistics  \nhttps://crsreports.congress.gov  | IF10244  · VERSION 68 · UPDATED   \n \nDisclaimer  \nThis document was  prepared by the Congressional Research Service (CRS). CRS serves as nonpartisan shared staff to \ncongressional committees and Members of Congress. It operates solely at the behest of and under the direction of Congress. \nInformation in a CRS Report should n ot be relied upon for purposes other than public understanding of information that has \nbeen provided by CRS to Members of Congress in connection with CRS’s institutional role. CRS Reports, as a work of the \nUnited States Government, are not subject to copyr ight protection in the United States. Any CRS Report may be \nreproduced and distributed in its entire

### Connect to local LLM, I'm using phi-3 from Microsoft

In [91]:
llm = Ollama(
    model="phi3",
    keep_alive=-1,
    format="json"
)

### Final output from the LLM using the context

In [92]:
llm.invoke(final_prompt)

'{"full_name": "National Interagency Coordination Center (NICC)"}'

In [84]:
import nltk  
from sklearn.feature_extraction.text import TfidfVectorizer  
from nltk.corpus import stopwords  
from sentence_transformers import SentenceTransformer  
import chromadb  
from chromadb.utils import embedding_functions  
  
# 初始化  
nltk.download("stopwords")  
nltk.download("punkt")  
stop_words = set(stopwords.words("english"))  
embedding_model = SentenceTransformer("Alibaba-NLP/gte-large-en-v1.5",trust_remote_code=True)  
client = chromadb.PersistentClient(path="chroma_data/")  
embedding_func = embedding_functions.SentenceTransformerEmbeddingFunction(  
    model_name="Alibaba-NLP/gte-large-en-v1.5", trust_remote_code=True  
)  
  
def preprocess_text(text):  
    tokens = nltk.word_tokenize(text)  
    tokens = [word.lower() for word in tokens]  
    tokens = [word for word in tokens if word.isalnum()]  
    tokens = [word for word in tokens if word not in stop_words]  
    return " ".join(tokens)  
  
def extract_keywords(text, n=5):  
    preprocessed_text = preprocess_text(text)  
    vectorizer = TfidfVectorizer()  
    tfidf_matrix = vectorizer.fit_transform([preprocessed_text])  
    feature_names = vectorizer.get_feature_names_out()  
    tfidf_scores = tfidf_matrix.toarray()[0]  
    keywords_scores = {feature_names[i]: tfidf_scores[i] for i in range(len(feature_names))}  
    sorted_keywords = sorted(keywords_scores.items(), key=lambda x: x[1], reverse=True)  
    return [keyword for keyword, score in sorted_keywords[:n]]  
  
def get_db_collection(collection_name: str) -> chromadb.Collection:  
    try:  
        collection = client.get_collection(  
            collection_name,  
            embedding_function=embedding_func,  
        )  
    except ValueError as e:  
        print(e)  
        collection = client.create_collection(  
            name=collection_name,  
            embedding_function=embedding_func,  
            metadata={"hnsw:space": "cosine"},  
        )  
    return collection  
  
def query_collection_combined(collection: chromadb.Collection, query_text: str, vector_weight=0.5, keyword_weight=0.5):  
    # 向量查询  
    vector_results = collection.query(query_texts=[query_text], n_results=2)  
      
    # 关键词提取  
    keywords = extract_keywords(query_text, n=5)  
    keyword_results = collection.query(query_texts=keywords, n_results=2)  
      
    # 合并结果  
    combined_results = {}  
      
    # 处理向量查询结果  
    for i, doc_id in enumerate(vector_results['ids'][0]):  
        score = vector_results['distances'][0][i] * vector_weight  
        if doc_id in combined_results:  
            combined_results[doc_id] += score  
        else:  
            combined_results[doc_id] = score  
      
    # 处理关键词查询结果  
    for i, doc_id in enumerate(keyword_results['ids'][0]):  
        score = keyword_results['distances'][0][i] * keyword_weight  
        if doc_id in combined_results:  
            combined_results[doc_id] += score  
        else:  
            combined_results[doc_id] = score  
      
    # 排序并返回前n个结果  
    sorted_results = sorted(combined_results.items(), key=lambda x: x[1], reverse=True)  
    top_results = sorted_results[:3]  
      
    # 获取详细信息  
    detailed_results = []  
    for doc_id, score in top_results:  
        index = vector_results['ids'][0].index(doc_id) if doc_id in vector_results['ids'][0] else keyword_results['ids'][0].index(doc_id)  
        document = vector_results['documents'][0][index] if doc_id in vector_results['ids'][0] else keyword_results['documents'][0][index]  
        metadata = vector_results['metadatas'][0][index] if doc_id in vector_results['ids'][0] else keyword_results['metadatas'][0][index]  
        detailed_results.append({  
            'id': doc_id,  
            'score': score,  
            'document': document,  
            'metadata': metadata  
        })  
      
    return detailed_results  
  
def generate_final_prompt(query_text, detailed_results):  
    context = "\n\n".join([f"Document ID: {result['id']}\nScore: {result['score']}\nContent: {result['document']}\nMetadata: {result['metadata']}" for result in detailed_results])  
    final_prompt = f"Query: {query_text}\n\nContext:\n{context}\n\nAnswer the query based on the above context."  
    return final_prompt  
  
# 示例用法  
if __name__ == "__main__":  
    collection = get_db_collection("my_project3")  
    query_text = "what is NICC full name?"  
    results = query_collection_combined(collection, query_text)  
      
    # 生成最终的提示  
    query_result = generate_final_prompt(query_text, results)  
    print(query_result)
      



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Query: what is NICC full name?

Context:
Document ID: docs-wildfire_stats-pdf-p2-c0
Score: 0.545694021650672
Content: Wildfire Statistics  
https://crsreports.congress.gov  | IF10244  · VERSION 68 · UPDATED   
 
Disclaimer  
This document was  prepared by the Congressional Research Service (CRS). CRS serves as nonpartisan shared staff to 
congressional committees and Members of Congress. It operates solely at the behest of and under the direction of Congress. 
Information in a CRS Report should n ot be relied upon for purposes other than public understanding of information that has 
been provided by CRS to Members of Congress in connection with CRS’s institutional role. CRS Reports, as a work of the 
United States Government, are not subject to copyr ight protection in the United States. Any CRS Report may be 
reproduced and distributed in its entirety without permission from CRS. However, as a CRS Report may include 
copyrighted images or material from a third party, you may need to o

In [85]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
).format(context=text)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
final_prompt = prompt.format(input=query_text)
final_prompt

"System: You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, say that you don't know. Use three sentences maximum and keep the answer concise.\n\nhttps://crsreports.congress.gov  \nUpdated June 1, 2023\nWildfire Statistics\nWildfires are unplanned fires, including lightning -caused \nfires, unauthorized human -caused fires, and escaped fires \nfrom prescribed burn projects. States are responsible for \nresponding to wildfires that begin o n nonfederal (state, \nlocal, and private) lands, except for lands protected by \nfederal agencies under cooperative agreements. The federal \ngovernment is responsible for responding to wildfires that \nbegin on federal lands. The Forest Service (FS) —within the \nU.S. Department of Agriculture —carries out wildfire \nmanagement and response across the 193 million acres of \nthe National Forest System  (NFS) . The Department of the \nInterior (DOI) mana

In [86]:
llm = Ollama(
    model="phi3",
    keep_alive=-1,
    format="json"
)

In [87]:
llm.invoke(final_prompt)

'{\n\n    "question": "what is NICC full name?",\n\n    "response": "The National Interagency Coordination Center (NICC) stands for the National Interagency Fire Center, which operates in Boise, Idaho. It\'s a hub where multiple federal agencies coordinate and manage wildland firefighting efforts across the United States.",\n\n    "source": "The name NICC is directly mentioned on page 4 of the provided document."\n\n}'

In [97]:
import nltk
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.llms import Ollama
from keyword_generator import extract_keywords
from db import get_db_collection, add_to_collection, query_collection
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.utils import embedding_functions

# 初始化
nltk.download("stopwords")
nltk.download("punkt")
stop_words = set(nltk.corpus.stopwords.words("english"))
embedding_model = SentenceTransformer("Alibaba-NLP/gte-large-en-v1.5", trust_remote_code=True)
client = chromadb.PersistentClient(path="chroma_data/")
embedding_func = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="Alibaba-NLP/gte-large-en-v1.5", trust_remote_code=True
)

# 加载PDF文档并分块
file_path = "docs/wildfire_stats.pdf"
loader = PyPDFLoader(file_path)
document = loader.load()
print("No. of pages in the document:", len(document))

text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=100)
chunked_documents = text_splitter.split_documents(document)

# 准备数据进行索引
contents = []
ids = []
keywords = []

page_no = 0
c_index = -1
for index, doc in enumerate(chunked_documents):
    metadata = doc.metadata
    source = metadata['source'].replace('/', '-').replace('.', '-')

    if metadata['page'] > page_no:
        c_index = 0
    else:
        c_index += 1

    page_no = metadata['page']
    chunk_id = f"{source}-p{page_no}-c{c_index}"

    contents.append(doc.page_content)
    ids.append(chunk_id)
    keywords.append(extract_keywords(doc.page_content))
    print("Processed chunk:", chunk_id)

# 创建Chroma DB集合并添加数据
COLLECTION_NAME = "my_project5"
collection = get_db_collection(COLLECTION_NAME)

metadata = [{"tags": ", ".join(i)} for i in keywords]
add_to_collection(collection, contents, ids, metadata)

# 混合查询函数
def query_collection_combined(collection, query_text, vector_weight=0.5, keyword_weight=0.5):
    # 向量查询
    vector_results = collection.query(query_texts=[query_text], n_results=5)

    # 关键词提取
    keywords = extract_keywords(query_text, n=5)
    keyword_results = collection.query(query_texts=keywords, n_results=5)

    # 合并结果
    combined_results = {}

    # 处理向量查询结果
    for i, doc_id in enumerate(vector_results['ids'][0]):
        score = vector_results['distances'][0][i] * vector_weight
        combined_results[doc_id] = combined_results.get(doc_id, 0) + score

    # 处理关键词查询结果
    for i, doc_id in enumerate(keyword_results['ids'][0]):
        score = keyword_results['distances'][0][i] * keyword_weight
        combined_results[doc_id] = combined_results.get(doc_id, 0) + score

    # 排序并返回前n个结果
    sorted_results = sorted(combined_results.items(), key=lambda x: x[1], reverse=True)
    top_results = sorted_results[:3]

    # 获取详细信息
    detailed_results = []
    for doc_id, score in top_results:
        index = vector_results['ids'][0].index(doc_id) if doc_id in vector_results['ids'][0] else keyword_results['ids'][0].index(doc_id)
        document = vector_results['documents'][0][index] if doc_id in vector_results['ids'][0] else keyword_results['documents'][0][index]
        metadata = vector_results['metadatas'][0][index] if doc_id in vector_results['ids'][0] else keyword_results['metadatas'][0][index]
        detailed_results.append({
            'id': doc_id,
            'score': score,
            'document': document,
            'metadata': metadata
        })

    return detailed_results

# 查询集合
query_text = "What are the components used in this project?"
results = query_collection_combined(collection, query_text)
print(results)

# 准备最终提示
text = ""
for result in results:
    text += result['document']

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
).format(context=text)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
final_prompt = prompt.format(input=query_text)
print(final_prompt)

# 连接本地LLM模型
llm = Ollama(
    model="phi3",
    keep_alive=-1,
    format="json"
)
response = llm.invoke(final_prompt)
print(response)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


No. of pages in the document: 3
Processed chunk: docs-wildfire_stats-pdf-p0-c0
Processed chunk: docs-wildfire_stats-pdf-p1-c0
Processed chunk: docs-wildfire_stats-pdf-p2-c0
Collection my_project5 does not exist.


Number of requested results 5 is greater than number of elements in index 3, updating n_results = 3


Documents loaded to DB


Number of requested results 5 is greater than number of elements in index 3, updating n_results = 3


[{'id': 'docs-wildfire_stats-pdf-p1-c0', 'score': 0.6433484590766934, 'document': 'Wildfire Statistics  \nhttps://crsreports.congress.gov  In 2022, 52% of the nationwide acreage burned by wildfires \nwas on federal lands ( 4.0 million acres ; see Table 1), lower \nthan the 10 -year average (64%) of impacted federal land \nacreage. The other 48% of the acreage burned in 2022 was \non state, local, or privately owned lands , though the f ires on \nthese lands  accounted for 83% of total fires. Of the federal \nacreage burned nationwide in 2022, 52% (2.1 million acres)  \nburned on DOI land and 47% (1.9 million acres) burned on \nFS land ( see Figure 3). The 2022 figures are driven largely \nby Alaska, where just over half of the acreage impacted \noccurred on nonfederal lands (1.6 million acres) and  just \nunder half was on DOI lands (1.5 million acres).  \nFigure 3. Percent age Acreage Burned by Owner ship \n \nSource:  NICC  Wildland Fire Summary and Statistics annual reports . \nNote

In [99]:
import nltk
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.llms import Ollama
from db import get_db_collection, query_collection
from keyword_generator import extract_keywords
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.utils import embedding_functions

# 初始化
nltk.download("stopwords")
nltk.download("punkt")
stop_words = set(nltk.corpus.stopwords.words("english"))
embedding_model = SentenceTransformer("Alibaba-NLP/gte-large-en-v1.5", trust_remote_code=True)
client = chromadb.PersistentClient(path="chroma_data/")
embedding_func = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="Alibaba-NLP/gte-large-en-v1.5", trust_remote_code=True
)

# 混合查询函数
def query_collection_combined(collection, query_text, vector_weight=0.5, keyword_weight=0.5):
    # 向量查询
    vector_results = collection.query(query_texts=[query_text], n_results=5)

    # 关键词提取
    keywords = extract_keywords(query_text, n=5)
    keyword_results = collection.query(query_texts=keywords, n_results=5)

    # 合并结果
    combined_results = {}

    # 处理向量查询结果
    for i, doc_id in enumerate(vector_results['ids'][0]):
        score = vector_results['distances'][0][i] * vector_weight
        combined_results[doc_id] = combined_results.get(doc_id, 0) + score

    # 处理关键词查询结果
    for i, doc_id in enumerate(keyword_results['ids'][0]):
        score = keyword_results['distances'][0][i] * keyword_weight
        combined_results[doc_id] = combined_results.get(doc_id, 0) + score

    # 排序并返回前n个结果
    sorted_results = sorted(combined_results.items(), key=lambda x: x[1], reverse=True)
    top_results = sorted_results[:3]

    # 获取详细信息
    detailed_results = []
    for doc_id, score in top_results:
        index = vector_results['ids'][0].index(doc_id) if doc_id in vector_results['ids'][0] else keyword_results['ids'][0].index(doc_id)
        document = vector_results['documents'][0][index] if doc_id in vector_results['ids'][0] else keyword_results['documents'][0][index]
        metadata = vector_results['metadatas'][0][index] if doc_id in vector_results['ids'][0] else keyword_results['metadatas'][0][index]
        detailed_results.append({
            'id': doc_id,
            'score': score,
            'document': document,
            'metadata': metadata
        })

    return detailed_results

# 查询集合
COLLECTION_NAME = "my_project5"
collection = get_db_collection(COLLECTION_NAME)
query_text = "As of June 1, 2023, how many wildfires have impacted"
results = query_collection_combined(collection, query_text)
print(results)

# 准备最终提示
text = ""
for result in results:
    text += result['document']

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
).format(context=text)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
final_prompt = prompt.format(input=query_text)
print(final_prompt)

# 连接本地LLM模型
llm = Ollama(
    model="phi3",
    keep_alive=-1,
    format="json"
)
response = llm.invoke(final_prompt)
print(response)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Number of requested results 5 is greater than number of elements in index 3, updating n_results = 3
Number of requested results 5 is greater than number of elements in index 3, updating n_results = 3


[{'id': 'docs-wildfire_stats-pdf-p2-c0', 'score': 0.47682925716207253, 'document': 'Wildfire Statistics  \nhttps://crsreports.congress.gov  | IF10244  · VERSION 68 · UPDATED   \n \nDisclaimer  \nThis document was  prepared by the Congressional Research Service (CRS). CRS serves as nonpartisan shared staff to \ncongressional committees and Members of Congress. It operates solely at the behest of and under the direction of Congress. \nInformation in a CRS Report should n ot be relied upon for purposes other than public understanding of information that has \nbeen provided by CRS to Members of Congress in connection with CRS’s institutional role. CRS Reports, as a work of the \nUnited States Government, are not subject to copyr ight protection in the United States. Any CRS Report may be \nreproduced and distributed in its entirety without permission from CRS. However, as a CRS Report may include \ncopyrighted images or material from a third party, you may need to obtain the permissio n of